# Querying ChEMBL

This notebook uses sqlite3 and pandas to query a local copy of the ChEMBL SQLite database, loading tables into pandas DataFrames and saving the data as csv files. For each target, we query ChEMBL for molecules with a reported activity against any of the ChEMBL IDs associated with that target. We apply the following criteria:

- Only measurements of Ki, Kd, IC50, or EC50 are used.
- No data validity comments are present.
- No 'approximate' measurements.
- No duplicate measurements (as identified by ChEMBL).
- High ChEMBL assay confidence score (>8).

This yields a respectable quantity of data for each target, and opens up some questions to explore. For example:

- Is it beneficial to use IC50/EC50 data interchangeably with Ki/Kd data (as is often the case in the literature), or is the benefit of increasing the size of the data set counteracted by the greater ambiguity in IC50/EC50 measurements?
- For similar targets (e.g. trypsin/thrombin and estrogen receptor 1/2), how many molecules interact with both targets, and how similar are the reported affinities?
- Is it easier to threshold the data and build a classification model to predict strong/weak binder than to build a regression model to predict binding affinity?
- Are there any outliers in the data set (see feature computation), and how might these affect model building?

We use the ChEMBL IDs associated with each target idenfied previously. Remember that there were no ChEMBL IDs for the target INHA.

In [1]:
import json
import sqlite3

import pandas as pd

with open('dude_target_chembl_ids.json') as f:
    target_chembl_ids = json.load(f)

[Here](https://chembl.gitbook.io/chembl-interface-documentation/frequently-asked-questions/schema-questions-and-sql-examples) is a good place to look for SQL examples using the ChEMBL schema.

In [2]:
# Change this to wherever your local copy of chembl lives
connection = sqlite3.connect("./chembl_27/chembl_27_sqlite/chembl_27.db")

# Query chembl for affinity data for each target, saving the resulting table as a csv file
for target in target_chembl_ids:

    # Convert our list of target ChEMBL IDs into a string to plug into the query text
    target_string =", ".join([f"'{id}'" for id in target_chembl_ids[target]])
    
    query = f"""
    SELECT
    activities.standard_value            AS standard_value,
    activities.standard_relation         AS standard_relation,
    activities.standard_type             AS standard_type,
    activities.pchembl_value             AS pchembl_value,
    molecule_hierarchy.parent_molregno   AS molregno,
    compound_structures.canonical_smiles AS canonical_smiles,
    molecule_dictionary.chembl_id        AS chembl_id
    FROM activities
    JOIN assays ON activities.assay_id = assays.assay_id
    JOIN target_dictionary ON assays.tid = target_dictionary.tid
    JOIN target_components ON target_dictionary.tid = target_components.tid
    JOIN component_class ON target_components.component_id = component_class.component_id
    JOIN protein_family_classification ON component_class.protein_class_id = protein_family_classification.protein_class_id
    JOIN molecule_dictionary ON activities.molregno = molecule_dictionary.molregno
    JOIN molecule_hierarchy ON molecule_dictionary.molregno = molecule_hierarchy.molregno
    JOIN compound_structures ON molecule_hierarchy.parent_molregno = compound_structures.molregno
    WHERE activities.standard_units = 'nM' AND
        activities.standard_type IN ('EC50', 'IC50', 'Ki', 'Kd') AND
        activities.data_validity_comment IS NULL AND
        activities.standard_relation IN ('=', '<') AND
        activities.potential_duplicate = 0 AND assays.confidence_score >= 8 AND
        target_dictionary.chembl_id IN ({target_string}) AND
        target_dictionary.target_type = 'SINGLE PROTEIN'
    """

    df = pd.read_sql_query(query, connection)
    df.set_index('chembl_id', inplace=True)
    df.to_csv(f"./chembl_data/{target}_extracted_chembl_data.csv")

connection.close()
